In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/PR_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1724057303,PR,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,7.212701e+14
1,3828657103,PR,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,7.206104e+14
2,4150097210,PR,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,7.203105e+14
3,5502967210,PR,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,7.202520e+14
4,7953047007,PR,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,7.206104e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1724057303,PR,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,7.212701e+14
1,3828657103,PR,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,7.206104e+14
2,4150097210,PR,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,7.203105e+14
3,5502967210,PR,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,7.202520e+14
4,7953047007,PR,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,7.206104e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
77,1365588509,PR,"Avenida Ponce de Leon #1155, SAN JUAN, PR",18.447281,-66.069884,NaN
84,6466228606,PR,"Carr. 5506 KM 1.8, Mercedita, PR",-25.252089,-52.021541,NaN
111,2954357201,PR,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,NaN
351,2942887210,PR,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,NaN
672,9390667707,PR,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,NaN
1052,6116877007,PR,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,NaN
1363,3391277310,PR,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,NaN
1654,9603427100,PR,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,NaN
1665,2372087203,PR,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,NaN
1777,9332887705,PR,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('72')] #PR
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/18 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

18


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12672/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
77,1365588509,PR,"Avenida Ponce de Leon #1155, SAN JUAN, PR",18.447281,-66.069884,[721270018001012]
84,6466228606,PR,"Carr. 5506 KM 1.8, Mercedita, PR",-25.252089,-52.021541,[]
111,2954357201,PR,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,[]
351,2942887210,PR,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,[]
672,9390667707,PR,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,[]
1052,6116877007,PR,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,[]
1363,3391277310,PR,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,[]
1654,9603427100,PR,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,[]
1665,2372087203,PR,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,[]
1777,9332887705,PR,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12672/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
77,1365588509,PR,"Avenida Ponce de Leon #1155, SAN JUAN, PR",18.447281,-66.069884,721270018001012
84,6466228606,PR,"Carr. 5506 KM 1.8, Mercedita, PR",-25.252089,-52.021541,NaN
111,2954357201,PR,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,NaN
351,2942887210,PR,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,NaN
672,9390667707,PR,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,NaN
1052,6116877007,PR,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,NaN
1363,3391277310,PR,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,NaN
1654,9603427100,PR,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,NaN
1665,2372087203,PR,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,NaN
1777,9332887705,PR,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
84,6466228606,PR,"Carr. 5506 KM 1.8, Mercedita, PR",-25.252089,-52.021541,NaN
111,2954357201,PR,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,NaN
351,2942887210,PR,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,NaN
672,9390667707,PR,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,NaN
1052,6116877007,PR,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,NaN
1363,3391277310,PR,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,NaN
1654,9603427100,PR,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,NaN
1665,2372087203,PR,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,NaN
1777,9332887705,PR,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,NaN
1799,2760167303,PR,"167 Paris Street, Hato Rey, PR",48.856614,2.352222,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
77,1365588509,721270018001012
84,6466228606,NaN
111,2954357201,NaN
351,2942887210,NaN
672,9390667707,NaN
1052,6116877007,NaN
1363,3391277310,NaN
1654,9603427100,NaN
1665,2372087203,NaN
1777,9332887705,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/PR_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,771939,1724057303,PR,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,7.212701e+14
1,771940,3828657103,PR,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,7.206104e+14
2,771941,4150097210,PR,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,7.203105e+14
3,771942,5502967210,PR,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,7.202520e+14
4,771943,7953047007,PR,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,7.206104e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,771939,1724057303,PR,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,7.212701e+14,NaN
1,771940,3828657103,PR,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,7.206104e+14,NaN
2,771941,4150097210,PR,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,7.203105e+14,NaN
3,771942,5502967210,PR,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,7.202520e+14,NaN
4,771943,7953047007,PR,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,7.206104e+14,NaN
...,...,...,...,...,...,...,...,...
2947,774886,6288858603,PR,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,7.212701e+14,NaN
2948,774887,8218768310,PR,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,7.202103e+14,NaN
2949,774888,9097578505,PR,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,7.211307e+14,NaN
2950,774889,9110348403,PR,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,7.212700e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,771939,1724057303,PR,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,7.212701e+14,NaN,721270096012001.0
1,771940,3828657103,PR,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,7.206104e+14,NaN,720610402001013.0
2,771941,4150097210,PR,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,7.203105e+14,NaN,720310501131002.0
3,771942,5502967210,PR,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,7.202520e+14,NaN,720252018001004.0
4,771943,7953047007,PR,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,7.206104e+14,NaN,720610402001013.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1724057303,PR,"PONCE DE LEON 1431 SUITE 701, SANTURCE, PR",18.388480,-66.062466,721270096012001.0
1,3828657103,PR,"35 Diana St, Amelia Industrial Park, GUAYNABO, PR",18.416696,-66.118672,720610402001013.0
2,4150097210,PR,"5900 AVE. ISLA VERDE L2 PMB, CAROLINA, PR",18.443043,-66.025696,720310501131002.0
3,5502967210,PR,"100 AVE. LUIS MUNOZ MARIN, CAGUAS, PR",18.217878,-66.030109,720252018001004.0
4,7953047007,PR,"5 Carr Km 6 Hm 2 Barrio Amelia, GUAYNABO, PR",18.419611,-66.114715,720610402001013.0
...,...,...,...,...,...,...
2947,6288858603,PR,300 Ave Rincon de Gautier Las Vistas Shopping ...,18.357168,-66.062152,721270100022014.0
2948,8218768310,PR,Barrio Buena Vista Parcelas VA Carr. 167 Ramal...,18.315721,-66.203868,720210322002004.0
2949,9097578505,PR,"1244 Ave Munoz Rivera Reparto Universitario, P...",18.000550,-66.620863,721130715003007.0
2950,9110348403,PR,"1762 Calle Loiza Esq. Diez de Andino, San Juan...",18.451450,-66.059777,721270015002000.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
84,6466228606,PR,"Carr. 5506 KM 1.8, Mercedita, PR",-25.252089,-52.021541,NaN
111,2954357201,PR,URB INDUSTRIAL SABANETA CALLE HUCAR EDIFICIO M...,6.150486,-75.616879,NaN
351,2942887210,PR,URB INDUSTRIAL SABANETA CALLE UCAR EDIFICIO M2...,6.150486,-75.616879,NaN
672,9390667707,PR,"1402 N CORINTH ST RD SUITE 113, DAllas, PR",32.772884,-96.779098,NaN
1052,6116877007,PR,"Urb. Parkville # 1 Mejico St., GUAYNABO, PR",23.634501,-102.552784,NaN
1363,3391277310,PR,"CARRETERA 108 INTERIOR KM 3.2 BO CUBA, MAYAGUE...",21.521757,-77.781167,NaN
1654,9603427100,PR,"Frances 39 Amelia Industrial Park, GUAYNABO, PR",46.227638,2.213749,NaN
1665,2372087203,PR,"4ta sec levittown , Centro Comercial Lago Vist...",30.460197,-97.988348,NaN
1777,9332887705,PR,"1 Northfield Plaza, Suite 300, Northfield, PR",42.107711,-87.773691,NaN
1799,2760167303,PR,"167 Paris Street, Hato Rey, PR",48.856614,2.352222,NaN


Convert to File

In [19]:
fips_PR = fips_merge_drop

In [20]:
fips_PR.to_csv("../../../data/state_data/geo/geo_fips/23/PR_fips_scraped.csv") 